In [2]:

import azureml.core
azureml.core.VERSION

'1.1.5'

In [3]:
import os
import shutil

from azureml.core.workspace import Workspace
from azureml.core import Experiment

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.dnn import PyTorch

In [4]:
ws = Workspace.get(name="Azure_projects", subscription_id='#subscriptions', resource_group='fastai-practise')

In [5]:

project_folder ='./Baby-Vibes'
os.makedirs(project_folder, exist_ok=True)

experiment_name = 'Baby-Vibes'
experiment=Experiment(ws,name=experiment_name)

In [6]:
shutil.copy('train.py', project_folder)

'./Baby-Vibes/train.py'

In [7]:
cluster_name="gpucluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target")
    
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6',
                                                           max_nodes=4)
    
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
estimator = PyTorch(source_directory=project_folder, 
              compute_target=compute_target, 
              entry_script = 'train.py', 
              use_gpu = True,                   
              pip_packages = ['fastai==1.0.60','pillow==5.4.1'])

WARNING - framework_version is not specified, defaulting to version 1.4.


In [9]:
run = experiment.submit(estimator)
run.wait_for_completion(show_output=True)

heel for nvidia-ml-py3 (setup.py): started
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=2af7c89287dc150ff2aa0b25b22c74b09373123e37269d82085d7191ab068bb4
  Stored in directory: /root/.cache/pip/wheels/7f/26/a3/33f2079871e2bebb3f53a2b21c3ec64129b8efdd18a6263a52
  Created wheel for pyyaml: filename=PyYAML-5.3-cp36-cp36m-linux_x86_64.whl size=44229 sha256=411812003220a1ba17313fa3900f911fae216429a871de96025c6a7efbcc9910
  Stored in directory: /root/.cache/pip/wheels/b1/86/0d/10e6c39d3a2b85ba807d7657ee80f08cc16c03f2aa2adf8e46
  Created wheel for bottleneck: filename=Bottleneck-1.3.2-cp36-cp36m-linux_x86_64.whl size=331501 sha256=5c4b5b2561bc05eea4109bb3ab307bef75e456876916bbc8bdafd327e31b2b0f
  Stored in directory: /root/.cache/pip/wheels/f7/a7/14/9be836efed01ac0eb3c125ac006c143b55ebf689269877d0e8
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=bfb20ff2f70f1355fecdd9c1a5bdbd6d02d98c831

{'runId': 'Baby-Vibes_1584370522_4e27467d',
 'target': 'gpucluster',
 'status': 'Completed',
 'startTimeUtc': '2020-03-16T15:13:03.439673Z',
 'endTimeUtc': '2020-03-16T15:17:09.416337Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '99d11128-1610-4e34-a8d4-5f21468b416b',
  'AzureML.DerivedImageName': 'azureml/azureml_d96d9f93a4166c6bdee76d16171db82b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'gpucluster',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSeconds': None,
  'nodeCount': 1,
  'environment': {'name': 'Experiment Baby-Vibes Environment',
   'version': 'Autosave_2020-03-16T14:55:25Z_4d3e54b6',
   'python': {'interpreterPath': 'python',


In [10]:
!pip freeze > requirements.txt